In [214]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd

In [215]:
# stop_factory = StopWordRemoverFactory()
# stopwords = stop_factory.get_stop_words()

In [201]:
df = pd.read_csv("jokowi.csv", encoding = "ISO-8859-1", names = ['tweet', 'sentiment'], na_values= ' ')
df.shape

(257, 2)

In [216]:
df[1:15]

,tweet,sentiment
1,RT @KontraS: Hai! Berjumpa lagi dengan KontraS...,1.0
2,RT @fadlizon: Bohong lagi bohong lagi. Mikrofo...,0.0
3,"RT @TheCebongers: Prawobo sentil Jokowi ""Ugal2...",0.0
4,RT @bangzul_1988: @JajangRidwan19 @Hadisang70 ...,NaN
5,RT @narkosun: Jokowi dibilang ugal2an. jadi yg...,0.0
6,RT @IreneViena: Jika saya dapat membuktikan ke...,1.0
7,"RT @fariji_lacak: Sdh Kubilang Dari ""AWAL. Dem...",0.0
8,RT @mkhumaini: Hampir genap empat tahun pemeri...,0.0
9,jokowi Siapa pun Cawapres intinya Jokowi haru...,1.0
10,#2019jokowilanjut;1,1.0


In [217]:
list(df)
df.set_index('tweet')


,sentiment
tweet,
PARAHNYA DIKASI TEPUK TANGAN LAGI BOS!!!;0,0.0
RT @KontraS: Hai! Berjumpa lagi dengan KontraS yang akan kembali 'nyicil' membahas Laporan Evaluasi 4 Tahun Kinerja Pemerintahan @Jokowi ââ¦;1,1.0
RT @fadlizon: Bohong lagi bohong lagi. Mikrofon dimatikan dulu dong P LBP. https://t.co/eeAUEIzLAc;0,0.0
"RT @TheCebongers: Prawobo sentil Jokowi ""Ugal2an. Lalu"" Bung Adian membalas bukan sentilan tapi pukulan telak ke ""Prabowo! Pemerintahan"" yg ugâ¦;0",0.0
RT @bangzul_1988: @JajangRidwan19 @Hadisang70 @budimandjatmiko Saya melihat dari dalam diri bung @budimandjatmiko ini beliau sgt ingin puâ¦,NaN
RT @narkosun: Jokowi dibilang ugal2an. jadi yg membangun infrastruktur bagi2 sertifikat lapangan kerja pembangkit listrik jalan tol beâ¦;0,0.0
RT @IreneViena: Jika saya dapat membuktikan keputusan pemerintah terkait takeover saham mayority freeport yg dilakukan sekarang adalah tindâ¦;1,1.0
"RT @fariji_lacak: Sdh Kubilang Dari ""AWAL. Demokrat PKS &amp; PAN, FOKUS pd PILLEG ketimbang PILPRES. 1.Mereka"" tdk ingin kehilangan Wakilnya diâ¦;0",0.0
RT @mkhumaini: Hampir genap empat tahun pemerintah mencatatkan totalÂ utangÂ mencapai Rp4.416 triliun naik sekitar Rp1.815 triliun dari posâ¦;0,0.0


In [47]:
df.columns[df.isnull().any()]

Index(['sentiment'], dtype='object')

In [204]:
new_df = df.fillna(method="ffill")
new_df

,tweet,sentiment
0,PARAHNYA DIKASI TEPUK TANGAN LAGI BOS!!!;0,0.0
1,RT @KontraS: Hai! Berjumpa lagi dengan KontraS...,1.0
2,RT @fadlizon: Bohong lagi bohong lagi. Mikrofo...,0.0
3,"RT @TheCebongers: Prawobo sentil Jokowi ""Ugal2...",0.0
4,RT @bangzul_1988: @JajangRidwan19 @Hadisang70 ...,0.0
5,RT @narkosun: Jokowi dibilang ugal2an. jadi yg...,0.0
6,RT @IreneViena: Jika saya dapat membuktikan ke...,1.0
7,"RT @fariji_lacak: Sdh Kubilang Dari ""AWAL. Dem...",0.0
8,RT @mkhumaini: Hampir genap empat tahun pemeri...,0.0
9,jokowi Siapa pun Cawapres intinya Jokowi haru...,1.0


In [205]:
sample_text = new_df.tweet[1]
sample_text 

"RT @KontraS: Hai! Berjumpa lagi dengan KontraS yang akan kembali 'nyicil' membahas Laporan Evaluasi 4 Tahun Kinerja Pemerintahan @Jokowi â\x80\x93â\x80¦;1"

In [206]:
def cleanningTweet(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT)|([:1])"," ",text).split())

x="RT @KontraS: Hai! Berjumpa lagi dengan KontraS yang akan kembali 'nyicil' membahas Laporan Evaluasi 4 Tahun Kinerja Pemerintahan @Jokowi â\x80\x93â\x80¦;1"

print(cleanningTweet(x))
cleaned_string = cleanningTweet(sample_text)

Hai Berjumpa lagi dengan KontraS yang akan kembali nyicil membahas Laporan Evaluasi 4 Tahun Kinerja Pemerintahan


##### Add stopwords removal

In [207]:
word_tokens = word_tokenize(cleaned_string)
filtered_sentence = [w for w in word_tokens if not w in stopwords]

resultStringTokenize = ' '.join(filtered_sentence)
resultStringTokenize

'Hai Berjumpa KontraS nyicil membahas Laporan Evaluasi 4 Tahun Kinerja Pemerintahan'

##### add more stopword removal 

In [209]:
# #Create factory
# stop_factory = StopWordRemoverFactory()
# more_stopword = ['hai','Hai','Hi','hi', 'ia','bahwa','oleh','adalah']
# # Tambahkan Stopword Baru
# data = stop_factory.get_stop_words()+more_stopword
 
# stopword = stop_factory.create_stop_word_remover()
# stopwordResult = stopword.remove(resultStringTokenize)
# stopwordResult

##### Stemming words

In [198]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
stemmerFactory = StemmerFactory()
stemmer = stemmerFactory.create_stemmer()
hasilStemmer = stemmer.stem(stopwordResult)
print(hasilStemmer)

hai jumpa kontras nyicil bahas lapor evaluasi 4 tahun kerja perintah


In [213]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'
Teks = nlp(stopwordResult)
# for word in Teks:
#     print(word.text, word.lemma_)
Token_kata = [token.lemma_ for token in Teks]
resultWithSpacy = ' '.join(Token_kata)
resultWithSpacy

'Hai Berjumpa KontraS nyicil bahas Laporan Evaluasi 4 Tahun Kinerja Pemerintahan'